Importing Libraries.

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
import pandas as pd 
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt
#%matplotlib inline  
import matplotlib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import numpy as np

from collections import defaultdict

from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from itertools import cycle
from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
#Learning a classification model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import seaborn # make prettier graphs
import re
#LDA
from gensim.models import LdaMulticore, TfidfModel, CoherenceModel
from gensim.corpora import Dictionary
import time # to know how long training took
import multiprocessing 
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline  
import matplotlib
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
import seaborn as sb

Let's load the datasets

In [ ]:
train = pd.read_csv('../input/spooky/train.csv')
test = pd.read_csv('../input/spooky/test.csv')
traincleaned=pd.read_csv('../input/dataclean/data.csv')
#traincleaned is the train data after cleaning, so I saved and load it here due to simplisity.

In [ ]:
#traincleaned[:3].drop(traincleaned.columns[1], axis=1)
traincleaned.drop('Unnamed: 0', axis=1,inplace=True)  
traincleaned[:3]

In [ ]:
traincleaned.shape

In [ ]:



def custom_colours(*args, **kwargs):
    word = args[0]  # First argument is the word itself
    if "o" in word: 
        return "#FFD700"
    return "#CCCCCC"

comment_words = '' 
stopwords = set(STOPWORDS) 
  
# iterate through the csv file 
for val in traincleaned.clean_text: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width = 800, height = 400, 
                stopwords = stopwords, 
                min_font_size = 8,background_color="white",colormap="winter").generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
#covid.drop(covid.columns[0], axis=1, inplace=True)
#covid.head(7)

#covid1=covid[:5000]
#covid1.shape

#covid1["author"]="covid"
#covid1["id"]="1"
#covid1

#data_merge=covid1.append(train, ignore_index=True)
#data_merge.head(2)
#data_merge.shape

A quick look at the data

In [ ]:
print(train.head(2))
print(test.head(1))

In [ ]:

stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
def preprocess(text, stem=False):
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  #text = re.sub(r"(.)\1+", "\\1", str(text).lower()).strip()
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        tokens.append(stemmer.stem(token))
      else:
        tokens.append(token)
  return " ".join(tokens)





#train['text'] =  train['text'].apply(str)
test['text'] =  test['text'].apply(str)

#train['text'] = train['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
test['text'] = test['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

#train['text'] = train['text'].str.replace("[^a-zA-Z#]", " ")
test['text'] = test['text'].str.replace("[^a-zA-Z#]", " ")

import re
#train.text = train.text.apply(lambda x: preprocess(x))
test.text = test.text.apply(lambda x: preprocess(x))




import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])

def clean_text(text):
    '''reduce text to lower-case lexicon entry'''
    lemmas = [token.lemma_ for token in nlp(text) 
              if token.pos_ in {'NOUN', 'VERB', 'ADJ', 'ADV', 'PROPN'}]
    return ' '.join(lemmas)






In [ ]:
#train['clean_text'] = train['text'].apply(clean_text)
#test['clean_text'] = test['text'].apply(clean_text)
#test['clean_text'].head()

In [ ]:
#path = "/Users/elhamnadimi/Downloads/nlp"

#test.to_csv('testclean.csv')

In [ ]:
train_sentence=traincleaned[:15663]
print(train_sentence.shape)
evalu = traincleaned.tail(3916)
print(evalu.shape)

In [ ]:
data_list=train_sentence.clean_text.apply(str).tolist()
test_list=evalu.clean_text.apply(str).tolist()
test_list[:2]

In [ ]:
data_list_1=traincleaned.clean_text.apply(str).tolist()

In [ ]:
train_sentence['clean_text'] = train_sentence['clean_text'].astype(str)
#train_sentence

In [ ]:
instances = traincleaned.clean_text.astype(str).apply(str.split)

instances.shape

In [ ]:
type(instances)

In [ ]:
instances2 = train_sentence.clean_text.astype(str).apply(str.split)

instances2.shape

In [ ]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:

def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(traincleaned['clean_text'].astype(str), 20)
#for word, freq in common_words:
    #print(word, freq)
df3 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df3.set_index("ReviewText", inplace = True)

#df3.groupby('ReviewText').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 bigrams in review before removing stop words')
plt.figure(figsize=(50,10)) 


df3.plot(kind="bar",width=0.5, color=(0.1, 0.1, 0.1, 0.4))
plt.title("Top 20 bigram",fontsize=15)
plt.xlabel("Bigrams",fontsize=15)
plt.ylabel("Frequency",fontsize=15)

fig = matplotlib.pyplot.gcf()
fig.set_size_inches(10.5, 5.5)

plt.xticks(fontsize=15)




In [ ]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(traincleaned['clean_text'].astype(str), 20)
#for word, freq in common_words:
#    print(word, freq)
df2 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df2.set_index("ReviewText", inplace = True)
#df3.groupby('ReviewText').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 bigrams in review before removing stop words')
plt.figure(figsize=(50,10)) 
df2.plot(kind="bar",width=0.5, color=(0.1, 0.1, 0.1, 0.4))
plt.title("Top 20 unigram",fontsize=15)
plt.xlabel("Unigram",fontsize=15)
plt.ylabel("Frequency",fontsize=15)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(10.5, 5.5)
plt.xticks(fontsize=15)

In [ ]:
tfidf = TfidfVectorizer(analyzer='word',
                        ngram_range=(1,2),min_df = 0.001,
                        max_df = 0.80,max_features = 8000,
                        stop_words = 'english',use_idf=True)
tfidf_results = tfidf.fit_transform(data_list_1)
print(tfidf_results.shape)



In [ ]:
x_tf = tfidf_results
y_tf = traincleaned["clean_text"].values
y=traincleaned["clean_text"].astype(str)

In [ ]:


selector = SelectKBest(chi2, k=1500).fit(tfidf_results,y)
X_sel = selector.transform(tfidf_results)
print(X_sel.shape)

In [ ]:
y.shape

In [ ]:
X_sel.shape

In [ ]:
print(tfidf.vocabulary_['bad'])
print(tfidf.get_feature_names()[905])
#print(len(train_sentence[train_sentence.clean_text.str.contains('bad')]))

In [ ]:
 # this also works with sparse matrices

# set number of latent components
k = 30

svd = TruncatedSVD(n_components=k)
%time U = svd.fit_transform(X_sel)
S = svd.singular_values_
V = svd.components_

In [ ]:
X_sel

In [ ]:
print(U.shape, S.shape, V.shape)

In [ ]:

def show_topics(A, vocabulary, topn=5):
    """
    find the top N words for each of the latent dimensions (=rows) in a
    """
    topic_words = ([[vocabulary[i] for i in np.argsort(t)[:-topn-1:-1]]
                    for t in A]) # for each row
    return [', '.join(t) for t in topic_words]

In [ ]:


t = V[0]
topn = 10
vocabulary = tfidf.get_feature_names()
[vocabulary[i] for i in np.argsort(t)[:-topn-1:-1]]

# t[1937]
# t[2366]

In [ ]:
#terms = tfidf.get_feature_names()

#show_topics(V, terms)

In [ ]:
# most important words for each topic
vocab = tfidf.get_feature_names()

for i, comp in enumerate(svd.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:5]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[
            0],end=" ")
    print("\n")

In [ ]:


sb.set_context('notebook')

def plot_vectors(vectors, title='VIZ', labels=None, dimensions=3):
    """
    plot the vectors in 2 or 3 dimensions. 
    If labels are supplied, use them to color the data accordingly
    """
    # set up graph
    fig = plt.figure(figsize=(10,10))

    # create data frame
    df = pd.DataFrame(data={'x':vectors[:,0], 'y': vectors[:,1]})
    # add labels, if supplied
    if labels is not None:
        df['label'] = labels
    else:
        df['label'] = [''] * len(df)

    # assign colors to labels
    cm = plt.get_cmap('tab10') # choose the color palette
    n_labels = len(df.label.unique())
    label_colors = [cm(1. * i/n_labels) for i in range(n_labels)]
    cMap = colors.ListedColormap(label_colors)
        
    # plot in 3 dimensions
    if dimensions == 3:
        # add z-axis information
        df['z'] = vectors[:,2]
        # define plot
        ax = fig.add_subplot(111, projection='3d')
        frame1 = plt.gca() 
        # remove axis ticks
        frame1.axes.xaxis.set_ticklabels([])
        frame1.axes.yaxis.set_ticklabels([])
        frame1.axes.zaxis.set_ticklabels([])

        # plot each label as scatter plot in its own color
        for l, label in enumerate(df.label.unique()):
            df2 = df[df.label == label]
            color_values = [label_colors[l]] * len(df2)
            ax.scatter(df2['x'], df2['y'], df2['z'], 
                       c=color_values, 
                       cmap=cMap, 
                       edgecolor='black', 
                       label=label, 
                       alpha=0.4, 
                       s=100)
      
    # plot in 2 dimensions
    elif dimensions == 2:
        ax = fig.add_subplot(111)
        frame1 = plt.gca() 
        frame1.axes.xaxis.set_ticklabels([])
        frame1.axes.yaxis.set_ticklabels([])

        for l, label in enumerate(df.label.unique()):
            df2 = df[df.label == label]
            color_values = [label_colors[l]] * len(df2)
            ax.scatter(df2['x'], df2['y'], 
                       c=color_values, 
                       cmap=cMap, 
                       edgecolor='black', 
                       label=label, 
                       alpha=0.4, 
                       s=100)

    else:
        raise NotImplementedError()

    plt.title(title)
#     plt.legend()
    plt.show()


In [ ]:
from sklearn.manifold import TSNE


Z = X_sel[:400].toarray()
tsne_lsa_model = TSNE(n_components=2, perplexity=50, learning_rate=100, 
                        n_iter=2000, verbose=1, random_state=0, angle=0.75)
tsne_lsa_vectors = tsne_lsa_model.fit_transform(Z)

In [ ]:


Z = X_sel[:500].toarray()
categories = traincleaned.author[:500]
X_embedded = TSNE(n_components=3).fit_transform(Z)
plot_vectors(X_embedded, title='TSNE', labels=categories, dimensions=2)

In [ ]:


num_clusters = 10
num_seeds = 10
max_iterations = 300
labels_color_map = {
    0: '#20b2aa', 1: '#ff7373', 2: '#ffe4e1', 3: '#005073', 4: '#4d0404',
    5: '#ccc0ba', 6: '#4700f9', 7: '#f6f900', 8: '#00f91d', 9: '#da8c49'
}
pca_num_components = 2
tsne_num_components = 2

# texts_list = some array of strings for which TF-IDF is being computed

# calculate tf-idf of texts
#tf_idf_vectorizer = TfidfVectorizer(analyzer="word", use_idf=True, smooth_idf=True, ngram_range=(2, 3))
#tf_idf_matrix = tf_idf_vectorizer.fit_transform(texts_list)

# create k-means model with custom config
clustering_model = KMeans(
    n_clusters=num_clusters,
    max_iter=max_iterations,
    precompute_distances="auto",
    n_jobs=-1
)

labels = clustering_model.fit_predict(X_sel[:4500])
# print labels

X = X_sel[:4500].todense()

# ----------------------------------------------------------------------------------------------------------------------

reduced_data = PCA(n_components=pca_num_components).fit_transform(X)
# print reduced_data
fig = plt.figure(figsize = (10, 10))
ax = plt.axes()
for index, instance in enumerate(reduced_data):
    # print instance, index, labels[index]
   
    
    pca_comp_1, pca_comp_2 = reduced_data[index]
    color = labels_color_map[labels[index]]
    ax.scatter(pca_comp_1, pca_comp_2, c=color)
    ax.legend()
    plt.title("Clustring topics -Kmean",fontsize=20)
    plt.xlabel("PCA 0",fontsize=20)
    plt.ylabel("PCA 1",fontsize=20)
    
plt.show()





    




In [ ]:
# t-SNE plot
#embeddings = TSNE(n_components=tsne_num_components)

#Y = embeddings.fit_transform(X[:500])
#plt.scatter(Y[:, 0], Y[:, 1], cmap=plt.cm.Spectral)
#plt.show()

In [ ]:
X_features = traincleaned.iloc[:,1:23]


In [ ]:


# read in instances and create Dictionary object w information about frequencies etc. 
dictionary = Dictionary(instances)
# get rid of words that are too rare or too frequent
dictionary.filter_extremes(no_below=50, no_above=0.5)
print(dictionary, flush=True)

In [ ]:
#replace words by their numerical IDs and their frequency
print("translating corpus to IDs", flush=True)
ldacorpus = [dictionary.doc2bow(text) for text in instances]
# learn TFIDF values from corpus
print("tf-idf transformation", flush=True)
tfidfmodel = TfidfModel(ldacorpus)
# transform raw frequencies into TFIDF
model_corpus = tfidfmodel[ldacorpus]

In [ ]:
print(instances[0])
print(ldacorpus[0]) 
print(model_corpus[0])

In [ ]:
#list(dictionary.items())

In [ ]:
coherence_values = []

dev_size = 14000
eval_size = 5000

for num_topics in range(5, 15):
    model = LdaMulticore(corpus=model_corpus[:dev_size], 
                         id2word=dictionary, 
                         num_topics=num_topics)

    coherencemodel_umass = CoherenceModel(model=model, 
                                          texts=instances[dev_size:dev_size+eval_size], 
                                          dictionary=dictionary, 
                                          coherence='u_mass')

    coherencemodel_cv = CoherenceModel(model=model, 
                                       texts=instances[dev_size:dev_size+eval_size], 
                                       dictionary=dictionary, 
                                       coherence='c_v')

    umass_score = coherencemodel_umass.get_coherence()
    cv_score = coherencemodel_cv.get_coherence()
    
    print(num_topics, umass_score, cv_score)
    coherence_values.append((num_topics, umass_score, cv_score))

In [ ]:

scores = pd.DataFrame(coherence_values, columns=['num_topics', 'UMass', 'CV'])

plt.figure(figsize=(50,10)) 
scores.plot(kind="line",x='num_topics',
                 y='UMass',marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=4)

plt.title("Topic coherance: Determining optimal number of topics ",fontsize=15)
plt.ylabel("Coherance score",fontsize=15)



In [ ]:

plt.figure(figsize=(50,10)) 
scores.plot(kind="line",x='num_topics', y='CV',marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=4)

plt.title("Topic coherance: Determining optimal number of topics ",fontsize=15)
plt.ylabel("Coherance score",fontsize=15)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(10.5, 5.5)


In [ ]:
num_topics = 6

# find chunksize to make about 200 updates
num_passes = 10
chunk_size = len(model_corpus) * num_passes/200
print(chunk_size)

start = time.time()
print("fitting model", flush=True)
model = LdaMulticore(num_topics=num_topics, # number of topics
                     corpus=model_corpus, # what to train on 
                     id2word=dictionary, # mapping from IDs to words
                     workers=min(10, multiprocessing.cpu_count()-1), # choose 10 cores, or whatever computer has
                     passes=num_passes, # make this many passes over data
                     chunksize=chunk_size, # update after this many instances
                     alpha=0.5
                    )
    
print("done in {}".format(time.time()-start), flush=True)

In [ ]:
# transform the data into topic distros
topic_corpus = model[model_corpus]

topic_corpus[0]

In [ ]:
model.print_topics()

In [ ]:


# get the topic descritions
topic_sep = re.compile(r"0\.[0-9]{3}\*") # getting rid of useless formatting
# extract a list of tuples with topic number and descriptors from the model
model_topics = [(topic_no, re.sub(topic_sep, '', model_topic).split(' + ')) for topic_no, model_topic in
                model.print_topics(num_topics=num_topics, num_words=5)]

descriptors = []
for i, m in model_topics:
    print(i+1, ", ".join(m[:10]))
    descriptors.append(", ".join(m[:2]).replace('"', ''))

In [ ]:
descriptors

In [ ]:
target_category = 'author'
limit = 1000
# get a list of all the topic scores for each document
scores = [[t[1] for t in topic_corpus[entry]] for entry in range(limit)]
# turn that into a data frame with N rows and K columns, each with the score of the corresponding topic
topic_distros = pd.DataFrame(data=scores, columns=descriptors)
# add the review category of each document (so we can aggregate)
topic_distros['category'] = train[target_category][:limit]

In [ ]:
topic_distros.head()

In [ ]:
import matplotlib.pyplot as plt # make graphs


seaborn.set_context('poster') # use large font

fig, ax = plt.subplots(figsize=(20, 10)) # set graph size


# aggregate topics by categories
aggregate_by_category = topic_distros[topic_distros.category.isin('EAP HPL MWS'.split())]
aggregate_by_category = aggregate_by_category.groupby(aggregate_by_category.category).mean()
# plot the graph
aggregate_by_category[descriptors].plot.bar(ax=ax,alpha=0.8,colormap="Set2");
# move the legend out
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5),fontsize=20);
ax.set_ylabel("Count",fontsize=25)
ax.set_xlabel("Authors",fontsize=25)
plt.title("Topics per Authors",fontsize=30)


In [ ]:
#Text classification :

In [ ]:


# transform labels into numbers
labels2numbers = LabelEncoder()

y1 = labels2numbers.fit_transform(train_sentence['author'])
print(train_sentence['author'][:10], y1[:10], len(y1))

In [ ]:
y1

In [ ]:
print(train_sentence.shape)
print(y1.shape)

In [ ]:
tfidf = TfidfVectorizer(analyzer='word',
                        ngram_range=(1,2),min_df = 0.001,
                        max_df = 0.80,max_features = 8000,
                        stop_words = 'english',use_idf=True)
tfidf_results_cl = tfidf.fit_transform(data_list)
print(tfidf_results_cl.shape)


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

selector = SelectKBest(chi2, k=1500).fit(tfidf_results_cl,y1)
X_sel_lg = selector.transform(tfidf_results_cl)
print(X_sel_lg.shape)

In [ ]:
labels2numbers.inverse_transform([1,1,1,0,0,1])

In [ ]:


classifier_lg = LogisticRegression(class_weight='balanced', solver='lbfgs', multi_class='ovr')
%time classifier_lg.fit(tfidf_results_cl , y1)
print(classifier_lg)

In [ ]:
coefs = classifier_lg.coef_
coefs

In [ ]:
k = tfidf.vocabulary_['bad'] # column position for the word
print(tfidf.get_feature_names()[k], classifier_lg.coef_[0, k])

In [ ]:


for k in [2,3,5,10,15,20]:
    cv = cross_val_score(LogisticRegression(), tfidf_results_cl, y=y1, cv=k, n_jobs=-1, scoring="accuracy")
    fold_size = tfidf_results_cl.shape[0]/k
    
    print("F1 with {} folds for bag-of-words is {}".format(k, cv.mean()))
    print("Training on {} instances/fold, testing on {}".format(fold_size*(k-1), fold_size))
    print()
    
    
scores = pd.Series(cv)
print(scores.min(), scores.mean(), scores.max())

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

most_frequent = DummyClassifier(strategy='most_frequent')

print(cross_val_score(most_frequent, tfidf_results_cl, y=y1, cv=5, n_jobs=-1, scoring="f1_micro").mean())

In [ ]:
y1.shape

In [ ]:
new_y = labels2numbers.transform(evalu['author'].astype(str))
new_y[:30]



In [ ]:
#evalu
new_X = tfidf.transform(evalu['clean_text'].astype(str))
new_y = labels2numbers.transform(evalu['author'].astype(str))



# use the old classifier to predict and evaluate
new_predictions = classifier_lg.predict(new_X)
print(new_predictions)

In [ ]:
#test_x = tfidf.transform(test['text'])
# use the old classifier to predict and evaluate
#test_predictions = classifier.predict(test_x)
#print(test_predictions)



In [ ]:
probabilities_lg = classifier_lg.predict_proba(new_X)
print(probabilities_lg)

In [ ]:
prediction_lg = classifier_lg.predict(new_X)
prediction_lg

In [ ]:
prediction_lg = classifier_lg.predict(new_X)
prediction_df = pd.DataFrame(data={'author': evalu['author'], 'new_predictions': labels2numbers.inverse_transform(prediction_lg),'truth':evalu['author']})
prediction_df[:5]

In [ ]:
#from sklearn.metrics import confusion_matrix
#import seaborn as sns

#Get the confusion matrix
#cf_matrix = confusion_matrix(new_y, new_predictions)
#sns.heatmap(cf_matrix, annot=True)
#sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
#            fmt='.2%', cmap='Blues')
#print(cf_matrix)




In [ ]:
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
new_y2=labels2numbers.inverse_transform(new_y)
new_y2
eval_predictions_lg=labels2numbers.inverse_transform(prediction_lg)
eval_predictions_lg




In [ ]:


cnf_matrix = confusion_matrix(new_y2, eval_predictions_lg,labels=['EAP', 'HPL', 'MWS'])
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['EAP', 'HPL', 'MWS'],
                      title='Confusion matrix')

In [ ]:
count_misclassified = (new_y != prediction_lg).sum()
print('Misclassified samples: {}'.format(count_misclassified))

accuracy = metrics.accuracy_score(new_y, prediction_lg)
print("\nPredicting with {:.2f}% accuracy".format(accuracy))

In [ ]:
y1[:7]

In [ ]:
new_y[:7]

In [ ]:

plt.style.use('ggplot')


ww = label_binarize(y1, classes=[0, 1, 2])
n_classes = ww.shape[1]
new_y22=label_binarize(new_y, classes=[0, 1, 2])

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(new_y22[:, i], probabilities_lg[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(new_y22.ravel(), probabilities_lg.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

# Plot ROC curve
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                   ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#Naive Bayes for Sentiment Analysis

In [ ]:


nb = MultinomialNB()
%time nb.fit(tfidf_results_cl, y1)
print(nb)

In [ ]:
coefnb = nb.coef_
coefnb

In [ ]:
k = tfidf.vocabulary_['bad'] # column position for the word
print(tfidf.get_feature_names()[k], nb.coef_[0, k])

In [ ]:
from sklearn.model_selection import cross_val_score

for k in [2,3,5,10,15,20]:
    cv2 = cross_val_score(MultinomialNB(), tfidf_results_cl, y=y1, cv=k, n_jobs=-1, scoring="f1_micro")
    fold_size = tfidf_results_cl.shape[0]/k
    
    print("F1 with {} folds for bag-of-words is {}".format(k, cv.mean()))
    print("Training on {} instances/fold, testing on {}".format(fold_size*(k-1), fold_size))
    print()
    
    scores = pd.Series(cv2)
print(scores.min(), scores.mean(), scores.max())

In [ ]:
# use the old classifier to predict and evaluate, evaluation set
predictions_nb = nb.predict(new_X)
print(predictions_nb)

In [ ]:
print(classification_report(new_y, predictions_nb))

In [ ]:
count_misclassified_nb = (new_y != predictions_nb).sum()
print('Misclassified samples: {}'.format(count_misclassified_nb))

accuracy_nb = metrics.accuracy_score(new_y, predictions_nb)
print("\nPredicting with {:.2f}% accuracy".format(accuracy_nb))

In [ ]:
#y_pred = model.predict(X_test)
#f1score = f1_score(y_test, y_pred)
#print(f"Tfidf Model Score: {f1score * 100} %")

In [ ]:
probabilities_nb = nb.predict_proba(new_X)
print(probabilities_nb)

In [ ]:
predictions_nb = nb.predict(new_X)
prediction_df = pd.DataFrame(data={'author': evalu['author'], 'new_predictions': labels2numbers.inverse_transform(predictions_nb),'truth':evalu['author']})
prediction_df[:5]

In [ ]:
predictions_nb

In [ ]:
new_predictions_nb_=labels2numbers.inverse_transform(predictions_nb)
new_predictions_nb_

In [ ]:
new_predictions_nb_

In [ ]:
cnf_matrix = confusion_matrix(new_y2, new_predictions_nb_,labels=['EAP', 'HPL', 'MWS'])
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['EAP', 'HPL', 'MWS'],
                      title='Confusion matrix')

In [ ]:
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from itertools import cycle
plt.style.use('ggplot')


ww = label_binarize(y1, classes=[0, 1, 2])
n_classes = ww.shape[1]
new_y22=label_binarize(new_y, classes=[0, 1, 2])

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=0)
#y_score = new_probabilities
plt.figure(figsize=(10,5)) 
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(new_y22[:, i], probabilities_nb[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
colors = cycle(['navy', 'aqua', 'deeppink'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=1.5,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=1.5)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curves for LogisticRegression')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Your code here
class LM:
 
  def __init__(self, smoothing, n_grams_size):
    self.smoothing = smoothing
    self.n_grams_size = n_grams_size
    self.counts = defaultdict(lambda: defaultdict(lambda: smoothing))
    self.START = '_***_'
    self.STOP = '_STOP_'

  def fit(self, document):
    """
    fit on a given document and collect counts
    """
    corpus = [line.strip().split() for line in document]

    for sentence in corpus:
      tokens = [self.START for _ in range(self.n_grams_size)] + sentence + [self.STOP]
      for n_grams_tuple in nltk.ngrams(tokens, self.n_grams_size):
        self.counts[n_grams_tuple[:-1]][n_grams_tuple[-1]] += 1
#[:-1] .It slices the string to omit the last character
  def _sample_next_word(self, history_tuple, iters=50):
    """
    sample next word based on the history
    """
    for _ in range(iters):
      keys, values = zip(*self.counts[history_tuple].items())
      values = np.array(values)
      values /= values.sum()
      sample = np.random.multinomial(1, values) 

    return keys[np.argmax(sample)]

  def generate(self, words_list):
    """
    generate a new sentence that begins with the given words
    """
    # check number of words
    if len(words_list) < self.n_grams_size:
      raise AttributeError("wrong number of words")

    result = [self.START for _ in range(self.n_grams_size)]
    result.extend(words_list)
    history_tuple = tuple([result[i] for i in range(-self.n_grams_size+1, 0, 1)])
    next_word = self._sample_next_word(history_tuple)
    result.append(next_word)

    while next_word != self.STOP:
      history_tuple = tuple(result[i] for i in range(-self.n_grams_size+1, 0, 1))
      next_word = self._sample_next_word(history_tuple)
      result.append(next_word)

    return ' '.join(result[self.n_grams_size:-1])

In [ ]:


tweets = [line.strip() for line in open('../input/spooky/train.csv', encoding='utf8')]
 
lm = LM(smoothing=0.001, n_grams_size=2)
lm.fit(document=tweets)
u=np.unique([lm.generate(["lovely","winter"]) for _ in range(10)])
test_subset=u.tolist()
test_subset

In [ ]:
import re
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('eap', '')
    text = text.replace('mws', '')


#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text


In [ ]:
map_object = map(clean_text, test_subset)
out_put = list(map_object)
out_put

In [ ]:
tokenized_sents = [word_tokenize(i) for i in out_put]
from itertools import chain
tokens_output = list(chain(*tokenized_sents))
tokens_output[:8]

In [ ]:
###################################

In [ ]:
traincleaned[:3]

In [ ]:
X = traincleaned.clean_text
Y = traincleaned.author

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

1. 1. The problem here is, since there are different numbers in the same column, the model will misunderstand the data to be in some kind of order, 0 < 1 < 2. But this isn’t the case at all. To overcome this problem, we use One Hot Encoder.



In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
integer_encoded1 = label_encoder.fit_transform(y_train)
integer_encoded2 = label_encoder.fit_transform(y_test)


le_name_mapping = dict(zip(label_encoder.transform(label_encoder.classes_),label_encoder.classes_))
print("Label Encoding Classes as ")
print(le_name_mapping)


#Converts a class vector (integers) to binary class matrix.
y_train=np_utils.to_categorical(integer_encoded1,num_classes=3)
y_val=np_utils.to_categorical(integer_encoded2,num_classes=3)
print("One Hot Encoded class shape ")
print(y_train.shape)
print(y_train[0])

In [ ]:
integer_encoded2

In [ ]:
y_val

In [ ]:
X_train0 = X_train.values.astype(str)
#y_train = df0_train['sentiment'].values

X_val0 = X_test.values.astype(str)
#y_val = df0_val['sentiment'].values
print('df_train shape: {}'.format(X_train0.shape))
print('df_val shape: {}'.format(X_val0.shape))

In [ ]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

# Vectorization parameters
# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Limit on the length of text sequences. Sequences longer than this
# will be truncated.
MAX_SEQUENCE_LENGTH = 500

#def sequence_vectorize(train_texts, val_texts):

    # Create vocabulary with training texts.
tokenizer = text.Tokenizer(num_words=TOP_K)
tokenizer.fit_on_texts(X_train0)

    # Vectorize training and validation texts.
x_train = tokenizer.texts_to_sequences(X_train0)
x_val = tokenizer.texts_to_sequences(X_val0)

    # Get max sequence length.
max_length = len(max(x_train, key=len))
if max_length > MAX_SEQUENCE_LENGTH:
    max_length = MAX_SEQUENCE_LENGTH

    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
x_train = sequence.pad_sequences(x_train, maxlen=max_length)
x_val = sequence.pad_sequences(x_val, maxlen=max_length)


vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index


print(x_train)
print(max_length)
print(x_train.shape)

print(vocab_size)
print(x_train[2])
print(X_train0[1])

In [ ]:
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences    
#tokenizer = Tokenizer(num_words=20000)
#tokenizer.fit_on_texts(X_train0)
#X_train = tokenizer.texts_to_sequences(X_train0)
#X_test = tokenizer.texts_to_sequences(X_val0)
#from keras.preprocessing.sequence import pad_sequences
#maxlen = 300
#padded_sequence_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
#padded_sequence_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
#print(vocab_size)
#print(X_train[1])
#print(X_train0[1])

In [ ]:
word_index = tokenizer.word_index


In [ ]:
len(tokenizer.word_index) + 1

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout,Bidirectional
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer


embedding_vector_length = 100

model = Sequential()

model.add(Embedding(len(tokenizer.word_index) + 1, embedding_vector_length,     
                                         input_length=409) )

#model.add(SpatialDropout1D(0.25))
model.add(LSTM(2, dropout=0.2, recurrent_dropout=0.3))

#model.add(Dropout(0.2))

model.add(Dense(3, activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
print(model.summary())


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout,Bidirectional
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer





embedding_vector_length = 100

model2 = Sequential(name="bi_lstm_20 NODES")

model2.add(Embedding(len(tokenizer.word_index) + 1, embedding_vector_length,     
                                         input_length=409,name="Bi-LSTM") )

model2.add(Bidirectional(LSTM(20, return_sequences=True, recurrent_dropout=0.4)))
model2.add(Dropout(0.5))
model2.add(Bidirectional(LSTM(20, recurrent_dropout=0.2)))

#model.add(Dropout(dropout))
#model2.add(Dense(256, activation='relu'))


#model.add(Dropout(0.2))

model2.add(Dense(3, activation='softmax'))
model2.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
print(model2.summary())



In [ ]:
x_train

In [ ]:
#history = model.fit(padded_sequence_train, y_train,validation_data=(padded_sequence_test, y_val),epochs=20,batch_size=70)

from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
history2=model2.fit(x_train, y_train,
 batch_size=500,
 epochs=20,
 validation_data=(x_val, y_val),
 callbacks=[es])
#We save this model so that we can use in own web app


In [ ]:
loss, accuracy = model2.evaluate(x_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model2.evaluate(x_val, y_val, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
y_score = model2.predict(x_val,verbose=1)
y_score

In [ ]:
y_score2 = model2.predict_classes(x_val,verbose=1)
y_score2

In [ ]:
y_val

In [ ]:
vv=np_utils.to_categorical(y_score2,num_classes=3)
vv

In [ ]:
probabilities = model2.predict_proba(x_val,verbose=1)
print(probabilities)

In [ ]:
prediction_model2 = model2.predict(x_val)
prediction_model2 = pd.DataFrame(data={'author': y_test, 'new_predictions': labels2numbers.inverse_transform(y_score2),'truth':y_test})
prediction_model2[:4]

In [ ]:

rr=labels2numbers.inverse_transform(integer_encoded2)

uu=labels2numbers.inverse_transform(y_score2)




cnf_matrix = confusion_matrix(rr, uu,labels=['EAP', 'HPL', 'MWS'])
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['EAP', 'HPL', 'MWS'],
                      title='Confusion matrix')

In [ ]:
eee=label_binarize(y_score2, classes=[0, 1, 2])
eee

In [ ]:
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from itertools import cycle
plt.style.use('ggplot')


#qqq = label_binarize(integer_encoded2, classes=[0, 1, 2])
n_classes = 3
eee=label_binarize(y_score2, classes=[0, 1, 2])

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=0)
#y_score = new_probabilities
plt.figure(figsize=(10,5)) 
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(eee[:, i], probabilities[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
colors = cycle(['navy', 'aqua', 'deeppink'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=1.5,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=1.5)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curves for LSTM')
plt.legend(loc="lower right")
plt.show()

In [ ]:

plt.style.use('ggplot')


qqq = label_binarize(integer_encoded2, classes=[0, 1, 2])
n_classes = qqq.shape[1]
eee=label_binarize(y_score2, classes=[0, 1, 2])

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(eee[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(eee.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

# Plot ROC curve
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                   ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
y_score
#The output for each sentence is a probability distribution over labels. Nice if we want it,
#but here, we are actually more interested in the one best tag for each token.

In [ ]:
#history = model.fit(padded_sequence_train, y_train,validation_data=(padded_sequence_test, y_val),epochs=20,batch_size=70)

from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
history=model.fit(x_train, y_train,
 batch_size=250,
 epochs=30,
 validation_data=(x_val, y_val),
 callbacks=[es])
#We save this model so that we can use in own web app
model.save('authors_res.h5')

In [ ]:
loss, accuracy = model.evaluate(x_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(x_val, y_val, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
plot_history(history2)

In [ ]:

plt.style.use('ggplot')


ww = label_binarize(y1, classes=[0, 1, 2])
n_classes = ww.shape[1]
new_y22=label_binarize(new_y, classes=[0, 1, 2])

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(new_y22[:, i], probabilities_lg[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(new_y22.ravel(), probabilities_lg.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

# Plot ROC curve
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                   ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
testdata = pd.read_csv('../input/testclean/testclean.csv')

testdata.shape

In [ ]:
testset_x= testdata.clean_text.astype(str)

In [ ]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text


TOP_K = 20000
MAX_SEQUENCE_LENGTH = 409

tokenizer = text.Tokenizer(num_words=TOP_K)
tokenizer.fit_on_texts(testset_x)
test_x0 = tokenizer.texts_to_sequences(testset_x)

max_length = len(max(testset_x, key=len))
if max_length > MAX_SEQUENCE_LENGTH:
    max_length = MAX_SEQUENCE_LENGTH

test_x0 = sequence.pad_sequences(test_x0, maxlen=max_length)

vocab_size2 = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index


print(max_length)
print(vocab_size2)



In [ ]:
y_test_pred = model.predict(test_x0,verbose=1)


In [ ]:
print(y_test_pred .shape)
print(testset_x[1])
print(y_test_pred[1])


In [ ]:
#lets use glove word embeding:

from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('../input/glove100/glove.6B.100d.txt', encoding="utf8")

We will create a dictionary that will contain words as keys and the corresponding 100 dimensional vectors as values, in the form of an array. Execute the following script:



In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

In [ ]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('../input/glove100/glove.6B.100d.txt', encoding='utf8')
for line in tqdm(f):
    values = line.split()
    word = ''.join(values[:-100])
    coefs = np.asarray(values[-100:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
words = ['sad']
for w in words:
    if w in embeddings_index.keys():
        print('Found the word {} in the dictionary'.format(w))

To feed this into an Embedding layer, we need to build a matrix containing the words in the data set and their representative word embedding. So this matrix will be of shape (NB_WORDS, GLOVE_DIM)

In [ ]:
NB_WORDS=17666 
GLOVE_DIM=100
emb_matrix = np.zeros((NB_WORDS, GLOVE_DIM))

for w, i in tokenizer.word_index.items():
    # The word_index contains a token for all words of the training data so we need to limit that
    if i < NB_WORDS:
        vect = embeddings_index.get(w)
        # Check if the word from the training data occurs in the GloVe word embeddings
        # Otherwise the vector is kept with only zeros
        if vect is not None:
            emb_matrix[i] = vect
    else:
        break

In [ ]:
emb_matrix.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers




embedding_vector_length = 100

model2 = Sequential()

model2.add(Embedding(17666, embedding_vector_length,     
                                         input_length=409) )

#model.add(SpatialDropout1D(0.25))
model2.add(LSTM(10, dropout=0.4, recurrent_dropout=0.3))
model2.add(Dropout(0.4))
#model2.add(layers.Flatten())
#model.add(Dense(units=output_labels))

model2.add(Dense(3, activation='softmax'))
model2.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
print(model2.summary())



In [ ]:
model2.layers[0].set_weights([emb_matrix])
model2.layers[0].trainable = False
print(model2.summary())



In [ ]:
#history = model.fit(padded_sequence_train, y_train,validation_data=(padded_sequence_test, y_val),epochs=20,batch_size=70)

from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
history=model2.fit(x_train, y_train,
 batch_size=400,
 epochs=30,
 validation_data=(x_val, y_val),
 callbacks=[es])
#We save this model so that we can use in own web app
model2.save('authors_res.h5')

In [ ]:
#using attention LSTM

In [ ]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding
from keras.layers import Bidirectional, LSTM
from keras.layers import Dropout, Dense, Activation
import numpy as np


features=409

inputs = Input((409, ), 
               name='word_IDs')
embeddings = Embedding(input_dim=len(tokenizer.word_index)+1, 
                       output_dim=32,name='embeddings',input_length=409)(inputs)
#wrap the LSTM in a Bidirectional wrapper
bilstm = LSTM(100,return_sequences=True,name="Bi-LSTM",dropout=0.4, recurrent_dropout=0.3)(embeddings)
att_out=attention()(bilstm)

output2 = Dense(3, activation='softmax')(att_out)

model_bilstm = Model(inputs=[inputs], outputs=[output2])

model_bilstm.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
model_bilstm.summary()

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
history=model_bilstm.fit(x_train, y_train,
 batch_size=300,
 epochs=30,
 validation_data=(x_val, y_val),
 callbacks=[es])
#We save this model so that we can use in own web app
model2.save('authors_res.h5')

In [ ]:
class Attention(tf.keras.Model):

	def __init__(self, units):
		super(Attention, self).__init__()
		self.W1 = tf.keras.layers.Dense(units)
		self.W2 = tf.keras.layers.Dense(units)
		self.V = tf.keras.layers.Dense(1)

	def call(self, features, hidden):
		# hidden shape == (batch_size, hidden size)
		# hidden_with_time_axis shape == (batch_size, 1, hidden size)
		# we are doing this to perform addition to calculate the score
		hidden_with_time_axis = tf.expand_dims(hidden, 1)
		  
		# score shape == (batch_size, max_length, 1)
		# we get 1 at the last axis because we are applying score to self.V
		# the shape of the tensor before applying self.V is (batch_size, max_length, units)
		score = tf.nn.tanh(
			self.W1(features) + self.W2(hidden_with_time_axis))
		# attention_weights shape == (batch_size, max_length, 1)
		attention_weights = tf.nn.softmax(self.V(score), axis=1)
		  
		# context_vector shape after sum == (batch_size, hidden_size)
		context_vector = attention_weights * features
		context_vector = tf.reduce_sum(context_vector, axis=1)
		return context_vector, attention_weights

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from collections import defaultdict
import re

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import Callback


maxlen=409
embed_size=100
max_features= len(tokenizer.word_index) +1

inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[emb_matrix])(inp)
lstm = Bidirectional(LSTM(50, return_sequences=True), name="bi_lstm_0")(x)

(lstm, forward_h, forward_c, backward_h, backward_c) = Bidirectional(LSTM(40,return_state=True,
                                                                          return_sequences=True),
                                                                          name="bi_lstm_1")(lstm)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
context_vector, attention_weights = Attention(20)(lstm, state_h)

#dense1 = Dense(20, activation="relu")(context_vector)

output = Dense(3, activation="softmax")(context_vector)
  
model = keras.Model(inputs=inp, outputs=output)

model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
history=model.fit(x_train, y_train,
 batch_size=300,
 epochs=30,
 validation_data=(x_val, y_val),
 callbacks=[es])
#We save this model so that we can use in own web app
model2.save('authors_res.h5')

In [ ]:
### from keras.layers import merge
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from collections import defaultdict
import re
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten,Permute,RepeatVector,Lambda
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import Callback
from keras import backend as K


maxlen=409
embed_size=100
max_features= len(tokenizer.word_index) +1

inp =Input(shape=(maxlen,), dtype='float32')
x = Embedding(max_features, embed_size, weights=[emb_matrix])(inp)
lstm = Bidirectional(LSTM(50, return_sequences=True), name="bi_lstm_0")(x)

attention = Dense(1, activation='tanh')(lstm)
attention= Flatten()(attention)
attention= Activation('softmax')(attention)
attention=RepeatVector(max_features*2)(attention)
attention= Permute([2,1])(attention)

sent_representation = Concatenate([lstm, attention])

sent_representation = Lambda(lambda xin: K.sum(xin, axis=-1))(sent_representation)

#output = Dense(3, activation="softmax")(sent_representation)
print (sent_representation.get_shape())    
#probabilities = Dense(3, activation='softmax')(sent_representation)      #Expected (5000,)
#model = Model(inputs=input_, outputs=probabilities)
#model.summary()



